Methods of Machine Learning and Intellectual Analysis of Data (Learning course) \
Bohdan Pavlyshenko (https://www.linkedin.com/in/bpavlyshenko/) \
Youtube video: https://www.youtube.com/watch?v=pZ_472c8AJM&list=PLMQt7tnruMvF1jetHNUKjMtHKSp78H2bb&index=14

# Text Classification (Pytorch, BERT)

Usefull resources:\
https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f \
https://github.com/marcellusruben/All_things_medium/blob/main/Text_Classification_BERT/bert_medium.ipynb \
https://towardsdatascience.com/multiclass-text-classification-using-lstm-in-pytorch-eac56baed8df \
https://jovian.ai/aakanksha-ns/lstm-multiclass-text-classification/v/5?utm_source=embed \
https://www.kaggle.com/code/yufengdev/bbc-text-categorization/data \
http://mlg.ucd.ie/datasets/bbc.html
https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

In [ ]:
#!pip install transformers

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels =  self.labels = [label_dict[label] for label in df['category'].tolist()]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        batch_texts = self.texts[idx]
        batch_y = np.array(self.labels[idx])

        return batch_texts, batch_y

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#path_drive = '/content/drive/MyDrive/Colab Notebooks/ml_course/'
#df = pd.read_csv(path_drive + 'data/data_iris.csv')
datapath = path_drive+'data/bbc-text.csv'
df = pd.read_csv(datapath)
df.head()

In [ ]:
datapath = 'data/bbc-text.csv'
df = pd.read_csv(datapath)
df.head()

In [ ]:
_=df.groupby(['category']).size().plot.bar()

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])
print(len(df_train),len(df_val), len(df_test))

len_df_train,len_df_val, len_df_test=len(df_train),len(df_val), len(df_test)

unique_labels = df_train.category.unique().tolist()
label_dict = {label:i  for i, label in enumerate(unique_labels)}
print ('label_dict',label_dict)

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
    
train, val = Dataset(df_train), Dataset(df_val)
train_dataloader = DataLoader(train, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val, batch_size=8)

test = Dataset(df_test)

test_dataloader = DataLoader(test, batch_size=2)

In [ ]:
tokenizer.vocab_size

In [ ]:
ds=Dataset(df_train)

In [ ]:
a=next(iter(ds))[0]['input_ids']
a

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
def train(model, train_dataloader, val_dataloader, learning_rate, epochs):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    
    model = model.to(device)
    criterion = criterion.to(device)
    
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            model.train()
        
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0
            
            model.eval()
            with torch.no_grad():

                len_val=0
                for val_input, val_label in val_dataloader:
                    len_val=len_val+1

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len_df_train: .3f} | Train Accuracy: {total_acc_train / len_df_train: .3f} | Val Loss: {total_loss_val / len_df_val: .3f} | Val Accuracy: {total_acc_val / len_df_val: .3f}')
                  

def evaluate(model, test_dataloader ):

    model = model.to(device)

    total_acc_test = 0
    
    model.eval()
    with torch.no_grad():

        len_test=0
        for test_input, test_label in test_dataloader:
            
            len_test=len_test+1

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    len_test=len_test*input_id.shape[0]    
    print(f'Test Accuracy: {total_acc_test / len_df_test: .3f}')


In [ ]:
EPOCHS = 3
model = BertClassifier()
LR = 1e-5

In [ ]:
train(model, train_dataloader, val_dataloader, LR, EPOCHS)

In [ ]:
evaluate(model, test_dataloader)